# API proposal for metric-learn to enhance scikit-learn compatibility

The following notebook will propose a draft of API for distance metric learning algorithms that are compatible with scikit-learn, allowing to easily do pipelines, cross-validations, and connect with other scikit-learn objects.

In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from sklearn.utils import check_random_state
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings('ignore')  # we still have warnings because of sparse arrays but will be fixed
from sklearn.pipeline import Pipeline, make_pipeline

# Description of the API

Many metric algorithms that are not supervised in the classical way (with inputs X and labels y) take as input some pairs where each of these has a label (1 or 0, for positive pairs/negative pairs also called positive/negative constraint). One might want to evaluate the performance of a metric learning algorithm by doing cross-validation of a score (roc_auc score for instance), splitting between train/test on the **constraints** and not on the points. Therefore to be able to do so, we could create a ``Pairs`` object that would contain the information of the points (``X``) **and** the pairs (that can be represented as two lists: the list of the first indexes of pairs ``a``, and the second indexes ``b``). This object would look like an array of couples of points, without replicating a point in the computer's memory for every constraint this point is involved in.

In [10]:
class Pairs():

    def __init__(self, X, a, b):
        self.a = a
        self.b = b
        self.X = X
        self.shape = (len(a), X.shape[1])

    def __getitem__(self, item):
        # Note that to avoid useless memory consumption, when splitting we delete the points that are not used
        a_sliced = self.a[item]
        b_sliced = self.b[item]
        unique_array = np.unique(np.concatenate([np.array(a_sliced), np.array(b_sliced)]))
        inverted_index = self._build_inverted_index(unique_array)
        pruned_X = self.X[unique_array].copy()  # copy so that the behaviour is always the same
        rescaled_sliced_a = inverted_index[a_sliced].A.ravel()
        rescaled_sliced_b = inverted_index[b_sliced].A.ravel()
        return Pairs(pruned_X, rescaled_sliced_a, rescaled_sliced_b)

    def __len__(self):
        return self.shape

    def __str__(self):
        return np.stack([self.X[self.a], self.X[self.b]], axis=1).__str__()
    
    def __repr__(self):
        return np.stack([self.X[self.a], self.X[self.b]], axis=1).__repr__()

    def asarray(self):
        return np.stack([self.X[self.a], self.X[self.b]], axis=1)

    @staticmethod
    def _build_inverted_index(unique_array):
        inverted_index = csr_matrix((np.max(unique_array) + 1, 1), dtype=int)
        inverted_index[unique_array] = np.arange(len(unique_array))[:, None]
        return inverted_index

Then we would want a MetricLearner to be able to train on such a ``Pairs`` object and on the labels of the corresponding ``Pairs``. We create a dummy metric learner just for testing purposes. It will just do one step in the direction of the gradient of pairwise distances of similar points w.r.t the metric matrix A. We will show how it can be used as a transformer as well as a classifier. 
- Its ``fit`` method takes as an input an object ``Pairs`` that represent pairs of points, and an array-like (or list like) ``y`` that represent the labels of the corresponding pairs (positive constraint of negative constraint). (so ``len(y) == len(a) == lenb(b) != len(X)``)
- Then when ``decision_function`` is called with input a ``Pairs`` object, the metric learner will return the pairwise distances of the considered pairs. This will be useful to evaluate the cross-validation roc_auc score when splitting train/test on the pairs.
- When ``transform`` is called with input a ``Pairs`` object, the metric learner will return a transformation of the **points** contained in the ``Pairs`` object. It can then return either a pairwise distance matrix on points, or an embedding of the points in the new space (if the algorithm can be expressed as such). This return type can be chosen at the creation of the Metric Learner by a flag.

Therefore this algorithm could be a classifier of the **constraints** as well as a transformer of the **points**, as we will see later.

In [3]:
class DummyMetricLearner(BaseEstimator, TransformerMixin):
    def __init__(self, return_embedding=True):
        self.A = None
        self.return_embedding = return_embedding
        
    def fit(self, pairs, y):
        X, constraints = self.prepare_input(pairs, y)
        diffs = X[constraints[0]] - X[constraints[1]]
        self.metric = diffs.T.dot(diffs)
    
    def fit_transform(self, pairs, y):
        self.fit(pairs, y)
        return self.transform(pairs)

    def decision_function(self, pairs):
        X_embedded = self.transform(pairs)
        squared_distances = np.sum((X_embedded[:, None] - X_embedded)**2,
                                   axis=2)
        return squared_distances[pairs.a, pairs.b]
    
    def transform(self, pairs):
        X_embedded = pairs.X.dot(self.metric)
        if self.return_embedding:
            return X_embedded
        else: 
            return np.sqrt(np.sum((X_embedded[:, None] - X_embedded)**2, axis=2))
    
    @staticmethod
    def prepare_input(X, y):
        a = X.a[y==0]
        b = X.b[y==0]
        c = X.a[y==1]
        d = X.b[y==1]
        X = X.X
        return X, [a, b, c, d]
        

# Examples

Let's create a very simple synthetic dataset of pairs and labels for the pairs, and then the ``Pairs`` object, as well a Metric Learner.

In [4]:
X = np.arange(0, 20)[:, None] * np.ones((20, 4))  # points
a = np.array([1, 2, 4, 6, 7, 10, 11, 14, 17, 10])  # first indices of pairs
b = np.array([5, 3, 6, 7, 1, 16, 14, 16, 17, 11])  # second indices of pairs
y = np.array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1])  # labels of the constraints

pairs = Pairs(X, a, b)
dml = DummyMetricLearner(return_embedding=True)

We can print pairs as we think of them: a list of couples of samples, in the form of a 3D numpy array.

In [5]:
pairs

array([[[  1.,   1.,   1.,   1.],
        [  5.,   5.,   5.,   5.]],

       [[  2.,   2.,   2.,   2.],
        [  3.,   3.,   3.,   3.]],

       [[  4.,   4.,   4.,   4.],
        [  6.,   6.,   6.,   6.]],

       [[  6.,   6.,   6.,   6.],
        [  7.,   7.,   7.,   7.]],

       [[  7.,   7.,   7.,   7.],
        [  1.,   1.,   1.,   1.]],

       [[ 10.,  10.,  10.,  10.],
        [ 16.,  16.,  16.,  16.]],

       [[ 11.,  11.,  11.,  11.],
        [ 14.,  14.,  14.,  14.]],

       [[ 14.,  14.,  14.,  14.],
        [ 16.,  16.,  16.,  16.]],

       [[ 17.,  17.,  17.,  17.],
        [ 17.,  17.,  17.,  17.]],

       [[ 10.,  10.,  10.,  10.],
        [ 11.,  11.,  11.,  11.]]])

We can do slicing on this object. It will indeed slice the pairs. This is useful for cross-validating metric learning algorithms.

In [6]:
pairs[2: 5]

array([[[ 4.,  4.,  4.,  4.],
        [ 6.,  6.,  6.,  6.]],

       [[ 6.,  6.,  6.,  6.],
        [ 7.,  7.,  7.,  7.]],

       [[ 7.,  7.,  7.,  7.],
        [ 1.,  1.,  1.,  1.]]])

We can now do what we want: a cross-validation of the roc-auc score of the metric learner, splitting on the pairs.

In [7]:
cross_val_score(dml, pairs, y, scoring='roc_auc', n_jobs=1)

array([ 0.  ,  0.75,  1.  ])

We can also do clustering: for instance with a KMeans that is trained on embeddings returned by a metric learner

In [8]:
from sklearn.cluster import KMeans
pipe = make_pipeline(dml, KMeans())
pipe.fit_predict(pairs, y)

array([3, 3, 7, 7, 1, 1, 1, 5, 5, 4, 4, 4, 6, 6, 0, 0, 0, 2, 2, 2], dtype=int32)

We can also chain a metric learner with unsupervised learning algorithms

In [9]:
from sklearn.decomposition import PCA
pipe_unsupervised = make_pipeline(dml, PCA(n_components=2))
pipe_unsupervised.fit_transform(pairs, y)

array([[  4.94000000e+03,   8.46302823e-13],
       [  4.42000000e+03,  -9.91412499e-14],
       [  3.90000000e+03,  -7.45049640e-14],
       [  3.38000000e+03,  -4.98686780e-14],
       [  2.86000000e+03,  -2.52323921e-14],
       [  2.34000000e+03,  -5.96106220e-16],
       [  1.82000000e+03,  -8.62270007e-14],
       [  1.30000000e+03,  -6.15907148e-14],
       [  7.80000000e+02,  -3.69544289e-14],
       [  2.60000000e+02,  -1.23181430e-14],
       [ -2.60000000e+02,   1.23181430e-14],
       [ -7.80000000e+02,   3.69544289e-14],
       [ -1.30000000e+03,   6.15907148e-14],
       [ -1.82000000e+03,   8.62270007e-14],
       [ -2.34000000e+03,   5.96106220e-16],
       [ -2.86000000e+03,   2.52323921e-14],
       [ -3.38000000e+03,   4.98686780e-14],
       [ -3.90000000e+03,   7.45049640e-14],
       [ -4.42000000e+03,   9.91412499e-14],
       [ -4.94000000e+03,   3.44311897e-13]])

<div class="alert alert-block alert-info">
Note that when creating a ``Pairs`` object, we keep the whole X inside it. This makes it possible to use the Metric Learner as a transformer on this X. However, when we do a splitting on the constraints, we will only keep the points from X that are present in the pairs of this slice, to be more memory efficient and to really create two datasets that are independent of one another. This is OK because ``Pairs`` splitting will be used only in the context of cross-validation on pairs and not samples, for algorithms that only look at the pairs.
</div>

# Discussion

This demo presents a simple possible API that allows to combine cross-validation scoring of the algorithm splitting on input pairs rather than points, while still allowing it to be used as a transformer in a pipeline. 
There are also other behaviours that are important in the API that are not discussed here:
- Like the current metric-learn API, there could be a supervised version of each semi-supervised algorithm (ex: MMC **and**  MMC_Supervised), that would do the job of creating constraints and calling the semi-supervised algorithm under the hood.
- Some algorithms should be able to be trained on constraints of type X[a] is more similar to X[b] than to X[c]
- Here we consider we have either labels (and we use supervised algorithms), or label of constraints (and we use semi-supervised algorithms). We never have both, and cannot do pipelines where the constraints information would help resolve a supervised problem with external labels. One could manually combine constraints and labels, and do the training without using pipeline/cross-validation framework, but we may think of helper functions and/or API features that would make this easier
- We could think of the hierarchy of objects, for instance supervised/[semi-supervised with pairs]/[other]..., [can return embedding]/[cannot] etc..., as well as the package organisation
- We should have a look at Pipe Graph  https://mail.python.org/pipermail/scikit-learn/2018-January/002158.html 